In [7]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [8]:
df_train = pd.read_csv("C:\\Users\\HUAWEI\\Downloads\\train.csv")
df_test = pd.read_csv("C:\\Users\\HUAWEI\\Downloads\\test.csv")

In [9]:
df_1 = df_train[df_train['label'] == 1]
df_0 = df_train[df_train['label'] == 0]
df_1 = df_1.sample(n=2000, frac=None, replace=False, weights=None, random_state=None, axis=None)
df_train = pd.concat([df_0,df_1],ignore_index = True)

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, output_size):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        out = out.squeeze(-1)
        return out

input_size = 4
hidden_size1 = 10
hidden_size2 = 10
output_size = 1
learning_rate = 0.1
batch_size = 128
num_epochs = 6

# generate data
X = df_train.iloc[:,:4]
y = df_train.iloc[:,4]

X = (X - X.mean())/X.std()

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state = 42)
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

X_unknown_test = df_test.iloc[:,:4]
X_unknown_test = X_unknown_test.to_numpy()
X_unknown_test = torch.tensor(X_unknown_test, dtype=torch.float32)
X_unknown_test = (X_unknown_test - X_unknown_test.mean())/X_unknown_test.std()

# create data loader
train_dataset = torch.utils.data.TensorDataset(X_train,y_train)
val_dataset =torch.utils.data.TensorDataset(X_test,y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

bn = nn.BatchNorm1d(num_features=4, track_running_stats=False)

# initialize model and optimizer
model = TwoLayerNet(input_size, hidden_size1,hidden_size2, output_size)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# define the loss function
criterion = nn.BCELoss()

accuracy = 0
for epoch in range(num_epochs):
    l= []
    for i, (inputs, labels) in enumerate(train_loader):
        # forward pass
        inputs = bn(inputs)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    for i, (inputs, labels) in enumerate(val_loader):  
        total_correct = 0
        total_samples = 0
        # test the model
        with torch.no_grad():
            inputs = bn(inputs)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            l.append(loss.item())
            predicted = (outputs > 0.5).float()
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    print(f"val,Epoch [{epoch+1}/{num_epochs}], Loss: {np.mean(l):.4f},Accuracy:{accuracy}")

val,Epoch [1/6], Loss: 0.6409,Accuracy:0.8125
val,Epoch [2/6], Loss: 0.4922,Accuracy:0.875
val,Epoch [3/6], Loss: 0.2645,Accuracy:1.0
val,Epoch [4/6], Loss: 0.1201,Accuracy:1.0
val,Epoch [5/6], Loss: 0.0642,Accuracy:1.0
val,Epoch [6/6], Loss: 0.0418,Accuracy:1.0


In [31]:
with torch.no_grad():
    X_unknown_test = bn(X_unknown_test)
    outputs = model(X_unknown_test)
    predicted = (outputs > 0.5).float()
    pp = predicted.tolist()

# Generalize result

test_id = df_test.iloc[:,4].to_numpy()

df_result = pd.DataFrame(test_id,columns = ['example_id'])
pp = list(map(int, pp[:]))
df_result['prediction'] = pp

df_result.to_csv('test_result.csv')

2936